In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

First I import the data as a dataframe

In [2]:
df = sqlContext.load(source="com.databricks.spark.csv", header = 'true', inferSchema = 'true', path = 'RandomForest_InputTestFile.csv')

/Users/arnaud/spark-1.6.0/python/pyspark/sql/context.py:538: UserWarning: load is deprecated. Use read.load() instead.
  warnings.warn("load is deprecated. Use read.load() instead.")


In [9]:
data = df.map(lambda (Y, X1, X2, X3, X4, X5, X6): LabeledPoint(Y, Vectors.dense([X1, X2, X3, X4, X5, X6]))).toDF()

Then I construct a very simple pipeline to train a `RandomForestClassifier`.

In [19]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

In [27]:
si = StringIndexer(inputCol="label", outputCol="Y")
rf = RandomForestClassifier(labelCol="Y")
pipeline = Pipeline(stages=[si, rf])

In [41]:
grid = ParamGridBuilder().addGrid(rf.maxDepth, range(20)).build()
evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator)

Finally, I train it and evaluate the best model using ROC (by default)

In [42]:
cvModel = cv.fit(data)

In [46]:
evaluator.evaluate(cvModel.transform(data))

0.9998622643460253